In [1]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime
# import hvplot.pandas
# import matplotlib.pyplot as plt
# from sklearn import svm
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from pandas.tseries.offsets import DateOffset
# from sklearn.metrics import classification_report

# from utils.helpers import alpaca_symbol_data

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
# print(sp500Tickers)

In [2]:
StocksWide = pd.read_sql_table('StocksWideFixed', engine)

In [3]:
StocksWide.drop(columns='index', inplace=True)

In [4]:
StocksWide = StocksWide.set_index('dateTime')


In [5]:
sectors = df500sectors['GICS Sector'].unique()
sectors

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Staples',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Real Estate', 'Energy'], dtype=object)

In [6]:
df500sectors['GICS Sector'].value_counts()

Information Technology    74
Industrials               73
Financials                66
Health Care               65
Consumer Discretionary    60
Consumer Staples          32
Real Estate               30
Utilities                 29
Materials                 28
Communication Services    26
Energy                    21
Name: GICS Sector, dtype: int64

In [7]:
SectorDF = StocksWide[['SPY']]
SectorDF

,SPY
dateTime,
2021-04-12 04:00:00,0.001898
2021-04-13 04:00:00,0.003134
2021-04-14 04:00:00,-0.003415
2021-04-15 04:00:00,0.005027
2021-04-16 04:00:00,-0.000072
...,...
2022-04-06 04:00:00,-0.000783
2022-04-07 04:00:00,0.006956
2022-04-08 04:00:00,-0.001049


In [8]:
stockstodrop = []
StocksWideNoSPY = StocksWide.drop(columns='SPY')
for sector in sectors:
    #print(sector)
    for (columnName, columnValues) in StocksWideNoSPY.iteritems():
        # print(df500sectors.loc[columnName]['GICS Sector'])
        if sector != df500sectors.loc[columnName]['GICS Sector']:
            # print(columnName)
            stockstodrop.append(columnName)
    TempDF = StocksWideNoSPY.drop(columns=stockstodrop)
    stockstodrop = []
    SectorDF[sector] = TempDF.mean(axis=1)


C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [11]:
SectorDF

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
dateTime,,,,,,,,,,,,
2021-04-12 04:00:00,0.001898,0.004364,0.001239,0.000170,0.002117,0.006057,0.001120,-0.000095,0.006061,0.002487,0.002266,-0.017691
2021-04-13 04:00:00,0.003134,-0.001641,0.006489,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2021-04-14 04:00:00,-0.003415,-0.000804,0.000607,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
2021-04-15 04:00:00,0.005027,0.000453,0.011968,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
2021-04-16 04:00:00,-0.000072,-0.003498,0.002734,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 04:00:00,-0.000783,0.002059,0.011836,-0.003392,-0.005008,0.009086,-0.006456,0.019110,-0.000604,-0.004417,0.014811,-0.010703
2022-04-07 04:00:00,0.006956,0.005856,0.020828,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626
2022-04-08 04:00:00,-0.001049,-0.005791,0.005329,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976


In [9]:
(SectorDF+1).cumprod()

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
dateTime,,,,,,,,,,,,
2021-04-12 04:00:00,1.001898,1.004364,1.001239,1.000170,1.002117,1.006057,1.001120,0.999905,1.006061,1.002487,1.002266,0.982309
2021-04-13 04:00:00,1.005039,1.002716,1.007736,1.000216,1.001348,1.006203,1.001118,1.015999,1.001803,0.999269,1.009967,0.983429
2021-04-14 04:00:00,1.001606,1.001910,1.008348,0.995598,0.996850,1.004596,0.998577,1.023856,1.012517,1.008544,1.004939,1.003434
2021-04-15 04:00:00,1.006641,1.002363,1.020416,1.000432,1.000256,1.009936,0.994048,1.035482,1.011421,1.012904,1.016911,0.996523
2021-04-16 04:00:00,1.006569,0.998857,1.023206,0.997739,0.996374,1.011708,0.996454,1.038314,1.007127,1.011386,1.013365,0.977631
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 04:00:00,1.027493,0.885173,1.123473,0.956374,0.865354,1.048806,0.832134,1.113349,0.928368,0.992611,1.078129,1.310163
2022-04-07 04:00:00,1.034640,0.890357,1.146872,0.961355,0.857274,1.057779,0.836041,1.109768,0.926995,0.996995,1.073324,1.313603
2022-04-08 04:00:00,1.033555,0.885200,1.152983,0.954490,0.858871,1.059778,0.839787,1.111231,0.929682,0.999008,1.075144,1.350352


In [10]:
SectorDF.to_sql('SectorDF', engine, if_exists='replace')

In [51]:
SP500_Constituents_details = pd.read_csv(Path("resources/constituents_financials.csv"), index_col='Symbol')
SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
SP500_Constituents_details.head()

,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
Symbol,,,,,,,,,,,,,
AAPL,Apple Inc.,Information Technology,155.15,16.86,1.579541,9.20,180.10,131.1200,8.095080e+11,7.938600e+10,3.458609,5.66,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOGL,Alphabet Inc Class A,Information Technology,1007.71,31.48,0.000000,22.27,1198.00,824.3000,7.338240e+11,3.421700e+10,6.801692,4.70,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOG,Alphabet Inc Class C,Information Technology,1001.52,40.29,0.000000,22.27,1186.89,803.1903,7.285360e+11,3.271400e+10,6.772653,4.67,http://www.sec.gov/cgi-bin/browse-edgar?action...
MSFT,Microsoft Corp.,Information Technology,85.01,25.76,1.874791,2.97,96.07,63.2200,6.899780e+11,4.107900e+10,7.113097,9.49,http://www.sec.gov/cgi-bin/browse-edgar?action...
AMZN,Amazon.com Inc,Consumer Discretionary,1350.50,296.16,0.000000,6.16,1498.00,812.5000,6.858730e+11,1.613200e+10,3.927053,24.28,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [52]:
StockCount = 0
Sleeve = 1
StockMarketCap = [[],[],[],[],[],[]]

for index, row in SP500_Constituents_details.iterrows():
    if StockCount == Sleeve * 100:
        Sleeve += 1
    StockMarketCap[Sleeve-1].append(index)
    StockCount += 1

In [53]:
StocksWide.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-04-12 04:00:00,-0.001293,-0.014194,0.021788,-0.007705,0.003988,0.008408,-0.005014,0.003981,0.005490,0.013559,...,-0.007185,-0.007863,0.009064,-0.022026,-0.002178,0.003796,0.012348,-0.002134,-0.003978,0.012701
2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,0.015665,...,0.001606,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684
2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,-0.006556,...,0.017394,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529
2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,0.014951,...,0.006271,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160
2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,0.002310,...,0.013402,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079


In [54]:
SleeveDf = StocksWide[['SPY']]
SleeveDf

,SPY
dateTime,
2021-04-12 04:00:00,0.001898
2021-04-13 04:00:00,0.003134
2021-04-14 04:00:00,-0.003415
2021-04-15 04:00:00,0.005027
2021-04-16 04:00:00,-0.000072
...,...
2022-04-06 04:00:00,-0.000783
2022-04-07 04:00:00,0.006956
2022-04-08 04:00:00,-0.001049


In [ ]:
SleeveDf = StocksWide[['SPY']]

for sleeve in StockMarketCap:
    for stock in sleeve:
        TempDF
        